Chapter 4 StanとRStanをはじめよう
===
---

## 4.4. 単回帰
データの分布の確認

In [1]:
# fig4-2.R
library(ggplot2)

d <- read.csv(file='input/data-salary.txt')
p <- ggplot(data=d, aes(x=X, y=Y))
p <- p + theme_bw(base_size=18)
p <- p + geom_point(shape=1, size=3)
ggsave(file='output/fig4-2.png', plot=p, dpi=300, w=4, h=3)
head(d)

X,Y
24,472
24,403
26,454
32,575
33,546
35,781


<img width="300" src="./output/fig4-2.png">

In [2]:
# lm.R (1)
d <- read.csv(file='input/data-salary.txt')
res_lm <- lm(Y ~ X, data=d)
res_lm # 切片と傾きが表示される


Call:
lm(formula = Y ~ X, data = d)

Coefficients:
(Intercept)            X  
     -119.7         21.9  


In [3]:
# lm.R(2)
X_new <- data.frame(X=23:60) # 23, 24, ..., 60を収めたdata frame
conf_95 <- predict(res_lm, X_new, interval='confidence', level=0.95)
pred_95 <- predict(res_lm, X_new, interval='prediction', level=0.95)

# 見てみる
head(conf_95)
head(pred_95)

,fit,lwr,upr
1,384.0995,309.3077,458.8913
2,406.0037,333.9625,478.0449
3,427.9079,358.5797,497.2361
4,449.8121,383.1547,516.4695
5,471.7163,407.6822,535.7504
6,493.6205,432.1561,555.0849


,fit,lwr,upr
1,384.0995,201.8638,566.3352
2,406.0037,224.8795,587.1279
3,427.9079,247.8456,607.9702
4,449.8121,270.7611,628.8631
5,471.7163,293.6253,649.8073
6,493.6205,316.4372,670.8038


In [4]:
# fig4-3.R
conf_95 <- predict(res_lm, X_new, interval='confidence', level=0.95)
conf_95 <- data.frame(X_new, conf_95)
conf_50 <- predict(res_lm, X_new, interval='confidence', level=0.50)
conf_50 <- data.frame(X_new, conf_50)
pred_95 <- predict(res_lm, X_new, interval='prediction', level=0.95)
pred_95 <- data.frame(X_new, pred_95)
pred_50 <- predict(res_lm, X_new, interval='prediction', level=0.50)
pred_50 <- data.frame(X_new, pred_50)

p <- ggplot()
p <- p + theme_bw(base_size=18)
p <- p + geom_ribbon(data=conf_95, aes(x=X, ymin=lwr, ymax=upr), alpha=1/6)
p <- p + geom_ribbon(data=conf_50, aes(x=X, ymin=lwr, ymax=upr), alpha=2/6)
p <- p + geom_line(data=conf_50, aes(x=X, y=fit), size=1)
p <- p + geom_point(data=d, aes(x=X, y=Y), shape=1, size=3)
p <- p + labs(x='X', y='Y') + coord_cartesian(xlim=c(22, 61), ylim=c(200, 1400))
p <- p + scale_y_continuous(breaks=seq(from=200, to=1400, by=400))
ggsave(p, file='output/fig4-3-left.png', dpi=300, w=4, h=3)

p <- ggplot()
p <- p + theme_bw(base_size=18)
p <- p + geom_ribbon(data=pred_95, aes(x=X, ymin=lwr, ymax=upr), alpha=1/6)
p <- p + geom_ribbon(data=pred_50, aes(x=X, ymin=lwr, ymax=upr), alpha=2/6)
p <- p + geom_line(data=pred_50, aes(x=X, y=fit), size=1)
p <- p + geom_point(data=d, aes(x=X, y=Y), shape=1, size=3)
p <- p + labs(x='X', y='Y') + coord_cartesian(xlim=c(22, 61), ylim=c(200, 1400))
p <- p + scale_y_continuous(breaks=seq(from=200, to=1400, by=400))
ggsave(p, file='output/fig4-3-right.png', dpi=300, w=4, h=3)

- 基本年収の信頼区間
<img width="400" src="./output/fig4-3-left.png">
- 年収の予測区間
<img width="400" src="./output/fig4-3-right.png">

### Stanで実装
model: model/model4-5.stan

In [5]:
# run-model4-5.R
library(rstan)

d <- read.csv(file='input/data-salary.txt')
data <- list(N=nrow(d), X=d$X, Y=d$Y) # stanに渡すデータを名前付きリストとして作成
fit <- stan(file='model/model4-5.stan', data=data, seed=1234) # モデルファイルとデータを渡して実行
# stan関数は内部でモデルファイルをコンパイルし(stan_model)、その後サンプリングを実行している(sampling)

# あとでdとfitを使用するために保存。R標準の機能
save.image(file='output/result-model4-5.RData')

Loading required package: StanHeaders
rstan (Version 2.16.2, packaged: 2017-07-03 09:24:58 UTC, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())


In file included from file69b47728b67e.cpp:8:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/src/stan/model/model_header.hpp:4:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/stan/math.hpp:4:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/stan/math/rev/mat.hpp:4:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/stan/math/rev/core.hpp:12:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/stan/math/rev/core/gevv_vvv_vari.hpp:5:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/stan/math/rev/core/var.hpp:7:
In file included from /usr/local/lib/R/3.4/site-library/BH/include/boost/math/tools/config.hpp:13:
In file included from /usr/local/lib/R/3.4/site-library/BH/include/boost/config.hpp:39:
/usr/local/lib/R/3.4/site-library/BH/include/boost/config/compiler/clang.hpp:200:11: warning: 'BOOST_NO_CXX11_RVALUE_REFERENCES' macro re

In [6]:
data
fit

$N
[1] 20

$X
 [1] 24 24 26 32 33 35 38 40 40 43 43 44 48 52 56 56 57 58 59 59

$Y
 [1]  472  403  454  575  546  781  750  601  814  792  745  837  868  988 1092
[16] 1007 1233 1202 1123 1314

Inference for Stan model: model4-5.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

         mean se_mean    sd    2.5%     25%     50%    75%  97.5% n_eff Rhat
a     -124.19    2.24 73.83 -270.62 -173.33 -125.17 -74.17  20.01  1085 1.01
b       22.01    0.05  1.65   18.86   20.88   22.00  23.09  25.25  1299 1.01
sigma   84.64    0.42 15.31   60.75   73.66   83.09  93.21 117.33  1342 1.00
lp__   -93.62    0.04  1.28  -96.78  -94.21  -93.32 -92.70 -92.17  1048 1.01

Samples were drawn using NUTS(diag_e) at Sat Nov 25 15:52:23 2017.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).

In [7]:
summary(fit)$summary

,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
a,-124.19266,2.24178810,73.830680,-270.62117,-173.32885,-125.16628,-74.16765,20.01005,1084.638,1.005302
b,22.00712,0.04568019,1.646096,18.86184,20.88007,22.00278,23.09480,25.25032,1298.538,1.005512
sigma,84.63697,0.41783045,15.305953,60.75277,73.65558,83.08910,93.20623,117.33481,1341.901,1.002162
lp__,-93.62346,0.03942713,1.276383,-96.78343,-94.21049,-93.32177,-92.70437,-92.16576,1048.024,1.005373


In [8]:
# rstan-save-diagnostics.R
library(ggmcmc) # ggplot2を利用した、MCMCの収束を診断するためのpackage

load('output/result-model4-5.RData')

# fitの要約を保存
write.table(data.frame(summary(fit)$summary),
  file='output/fit-summary.txt', sep='\t', quote=FALSE, col.names=NA)

# fitをもとにtrace plotを出力(pdf)
ggmcmc(ggs(fit, inc_warmup=TRUE, stan_include_auxiliar=TRUE),
  file='output/fit-traceplot.pdf', plot='traceplot')

ggmcmc(ggs(fit), file='output/fit-ggmcmc.pdf')
# 出力を一部に限定するには以下のようにする
# ggmcmc(ggs(fit), file='output/fit-ggmcmc.pdf',
#   plot=c('traceplot', 'density', 'running', 'autocorrelation'))

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: tidyr

Attaching package: ‘tidyr’

The following object is masked from ‘package:rstan’:

    extract



Plotting traceplots
Time taken to generate the report: 1 seconds.
Plotting histograms
Plotting density plots
Plotting traceplots
Plotting running means
Plotting comparison of partial and full chain
Plotting autocorrelation plots
Plotting crosscorrelation plot
Plotting Potential Scale Reduction Factors
Plotting Geweke Diagnostic
Plotting caterpillar plot
Time taken to generate the report: 4.7 seconds.


### MCMCの設定の変更

In [9]:
# rstan-modify-MCMCsettings.R
# stan関数を以下のようにstan_model とsamplingに分けることができる

stanmodel <- stan_model(file='model/model4-5.stan') # モデルファイルをコンパイル

fit <- sampling(
  stanmodel,
  data=data,
  pars=c('b', 'sigma'),# 保存する変数を限定できる
  init=function() {
    list(a=runif(1,-10,10), b=runif(1,0,10), sigma=10) # 初期値の設定
  },
  seed=123, # 乱数のseed: 解析の再現性のため
  chains=3, iter=1000, warmup=200, thin=2
  # chains: 最低3, 4以上を推奨
  # iter: 試行錯誤は500~1000ぐらい、モデルがきまったら1000~5000ぐらい
  # warmup: trace plotを見て決める
  # thin: 通常1でok. trace plotがあるパラメータで急激に大きくなったり小さくなったりする場合は5などにして自己相関を減らし、収束がよくなることがある
)


SAMPLING FOR MODEL 'model4-5' NOW (CHAIN 1).

Gradient evaluation took 2.6e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.26 seconds.
Adjust your expectations accordingly!


Iteration:   1 / 1000 [  0%]  (Warmup)
Iteration: 100 / 1000 [ 10%]  (Warmup)
Iteration: 200 / 1000 [ 20%]  (Warmup)
Iteration: 201 / 1000 [ 20%]  (Sampling)
Iteration: 300 / 1000 [ 30%]  (Sampling)
Iteration: 400 / 1000 [ 40%]  (Sampling)
Iteration: 500 / 1000 [ 50%]  (Sampling)
Iteration: 600 / 1000 [ 60%]  (Sampling)
Iteration: 700 / 1000 [ 70%]  (Sampling)
Iteration: 800 / 1000 [ 80%]  (Sampling)
Iteration: 900 / 1000 [ 90%]  (Sampling)
Iteration: 1000 / 1000 [100%]  (Sampling)

 Elapsed Time: 0.059597 seconds (Warm-up)
               0.043703 seconds (Sampling)
               0.1033 seconds (Total)


SAMPLING FOR MODEL 'model4-5' NOW (CHAIN 2).

Gradient evaluation took 6e-06 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
Adjust your 

### 並列計算の実行方法
MCMCはchainごとに独立している。並列化するとサンプリングの進捗状況やエラーメッセージが確認できなくなるので、並列化なしでうまくいくのを確認した上で並列化するのが良い。

In [10]:
rstan_options(auto_write=TRUE) # 再コンパイルせずに済むようにコンパイル後のモデルファイルを保存
options(mc.cores=parallel::detectCores()) # 複数のchainを並列で計算

### ベイズ信頼区間とベイズ予測区間の算出
- ベイズ信頼区間：パラメータの事後分布の両端から$\alpha %$の面積を切り捨てた区間が$(1-\alpha) %$ベイズ信頼区間
- ベイズ予測区間：ベイズ推定で求めた予測分布（パラメータの事後分布で重み付けしたデータの分布）から算出した予測区間

基本年収のベイズ信頼区間と年収のベイズ予測区間をStanの結果から計算する

In [11]:
# rstan-extract-MCMCsamples.R (1)

# MCMCサンプルを読み出す
load('output/result-model4-5.RData')
ms <- rstan::extract(fit) # extract()は他のパッケージにも存在するので名前空間を指定

In [12]:
quantile(ms$b, probs=c(0.025, 0.975)) # ms$bは数値arrayなのでquantileで簡単に95%区間を求めることができる

2.5%    97.5% 
18.86184 25.25032

MCMCサンプルは事後分布からサンプリングしたもので、各パラメータの同時分布になっている。

散布図を書いてパラメータの相関を見てみる

In [13]:
# fig4-7.R
library(ggplot2)
source('../common.R')

load('output/result-model4-5.RData')
ms <- rstan::extract(fit)
df_mcmc <- data.frame(a=ms$a, b=ms$b, sigma=ms$sigma)

p_xy <- ggplot(df_mcmc,aes(x=a,y=b)) +
  my_theme() +
  geom_point(alpha=1/4, size=2, shape=1) +
  scale_x_continuous(breaks=seq(-400, 200, 200), limits=c(-420, 210)) +
  scale_y_continuous(breaks=seq(15, 25, 5), limits=c(14.5, 29))

p_x <- ggplot(df_mcmc, aes(x=a)) +
  my_theme0() +
  geom_histogram(aes(y=..density..), colour='black', fill='white') +
  geom_density(alpha=0.3, fill='gray20') +
  scale_x_continuous(breaks=seq(-400, 200, 200), limits=c(-420, 210)) +
  labs(x='', y='')

p_y <- ggplot(df_mcmc, aes(x=b)) +
  my_theme0() +
  coord_flip() +
  geom_histogram(aes(y=..density..), colour='black', fill='white') +
  geom_density(alpha=0.3, fill='gray20') +
  scale_x_continuous(breaks=seq(15, 25, 5), limits=c(14.5, 29)) +
  labs(x='', y='')

p_emp <- ggplot(data.frame(0,0)) + theme_bw() + theme(rect=element_rect(fill='white'), panel.border=element_blank())

g_xy <- ggplotGrob(p_xy)
g_x <- ggplotGrob(p_x)
g_y <- ggplotGrob(p_y)
g_emp <- ggplotGrob(p_emp)

g1 <- cbind(g_x, g_emp, size='first')
g2 <- cbind(g_xy, g_y, size='first')
g <- rbind(g1, g2, size='first')
g$widths[1:3] <- grid::unit.pmax(g1$widths[1:3], g2$widths[1:3])
g$heights[7:12] <- g$widths[5:10] <- rep(unit(0.5,'mm'), 6)
g$heights[6] <- g$widths[11] <- unit(3,'cm')

png(file='output/fig4-7.png', res=300, w=1800, h=1800)
grid::grid.draw(g)
dev.off()

`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
Warning message:
“Removed 1 rows containing missing values (geom_bar).”`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.


pdf 
  2

<img width="400" src="./output/fig4-7.png">

同時分布を踏まえて50歳の人の基本年収の分布と年収の予測分布を作成

In [14]:
# # rstan-extract-MCMCsamples.R (2)
N_mcmc <- length(ms$lp__) # MCMCサンプルの長さ
y50_base <- ms$a + ms$b * 50 # 50歳の人の基本年収の分布からのMCMCサンプル
y50 <- rnorm(n=N_mcmc, mean=y50_base, sd=ms$sigma) # モデル式に従って正規分布に従う乱数を発生させ、年収の予測分布からのMCMCサンプルを作成
d_mcmc <- data.frame(a=ms$a, b=ms$b, sigma=ms$sigma, y50_base, y50) # データフレームを作成

In [15]:
quantile(d_mcmc$y50_base, probs=c(0.025, 0.975))
quantile(d_mcmc$y50, probs=c(0.025, 0.975))

2.5%     97.5% 
 933.2229 1019.9180

2.5%     97.5% 
 790.6927 1147.0635

上記を50歳以外の各年齢について行い、基本年収のベイズ信頼区間と年収のベイズ予測区間をプロットする。

Rのlmを使用した時とほぼ同じになっていることに注目！

In [16]:
# fig4-8.R
library(ggplot2)
source('../common.R')

load('output/result-model4-5.RData')
ms <- rstan::extract(fit)

X_new <- 23:60
N_X <- length(X_new)
N_mcmc <- length(ms$lp__)

set.seed(1234)
y_base_mcmc <- as.data.frame(matrix(nrow=N_mcmc, ncol=N_X))
y_mcmc <- as.data.frame(matrix(nrow=N_mcmc, ncol=N_X))
for (i in 1:N_X) {
  y_base_mcmc[,i] <- ms$a + ms$b * X_new[i]
  y_mcmc[,i] <- rnorm(n=N_mcmc, mean=y_base_mcmc[,i], sd=ms$sigma)
}

customize.ggplot.axis <- function(p) {
  p <- p + labs(x='X', y='Y')
  p <- p + scale_y_continuous(breaks=seq(from=200, to=1400, by=400))
  p <- p + coord_cartesian(xlim=c(22, 61), ylim=c(200, 1400))
  return(p)
}

d_est <- data.frame.quantile.mcmc(x=X_new, y_mcmc=y_base_mcmc)
p <- ggplot.5quantile(data=d_est)
p <- p + geom_point(data=d, aes(x=X, y=Y), shape=1, size=3)
p <- customize.ggplot.axis(p)
ggsave(file='output/fig4-8-left.png', plot=p, dpi=300, w=4, h=3)

d_est <- data.frame.quantile.mcmc(x=X_new, y_mcmc=y_mcmc)
p <- ggplot.5quantile(data=d_est)
p <- p + geom_point(data=d, aes(x=X, y=Y), shape=1, size=3)
p <- customize.ggplot.axis(p)
ggsave(file='output/fig4-8-right.png', plot=p, dpi=300, w=4, h=3)

- 基本年収のベイズ信頼区間
<img width="400" src="./output/fig4-8-left.png">
- 年収のベイズ予測区間
<img width="400" src="./output/fig4-8-right.png">

### transformed parametersブロックとgenerated quantitiesブロック
Stanで求めたMCMCサンプルを使ってRで様々な量を計算してきたが、Stan側でできるだけ処理を実行してしまいたい。

transformed parametersブロックは、他にもデータやパラメータを組み合わせて新しいパラメータ変数を作る場合にも使う。

修正したmodelはmodel4-4.stan参照

In [17]:
# run-model4-4.R
# データの読み込みから描画までをまとめて行う

# データの読み込み・stanの実行
d <- read.csv(file='input/data-salary.txt')
X_new <- 23:60
data <- list(N=nrow(d), X=d$X, Y=d$Y, N_new=length(X_new), X_new=X_new)
fit <- stan(file='model/model4-4.stan', data=data, seed=1234)
ms <- rstan::extract(fit)

# 分位点を取り出す関数
data.frame.quantile.mcmc <- function(x, y_mcmc,
  probs=c(2.5, 25, 50, 75, 97.5)/100) {
  qua <- apply(y_mcmc, 2, quantile, probs=probs)
  d <- data.frame(X=x, t(qua))
  colnames(d) <- c('X', paste0('p', probs*100))
  return(d)
}

# 50%区間をプロットする関数
ggplot.5quantile <- function(data) {
  p <- ggplot(data=data, aes(x=X, y=p50))
  p <- p + theme_bw(base_size=18)
  p <- p + geom_ribbon(aes(ymin=p2.5, ymax=p97.5), fill='black', alpha=1/6)
  p <- p + geom_ribbon(aes(ymin=p25, ymax=p75), fill='black', alpha=2/6)
  p <- p + geom_line(size=1)
  return(p)
}

# ggplotの軸をカスタマイズする関数
customize.ggplot.axis <- function(p) {
  p <- p + labs(x='X', y='Y')
  p <- p + scale_y_continuous(breaks=seq(from=200, to=1400, by=400))
  p <- p + coord_cartesian(xlim=c(22, 61), ylim=c(200, 1400))
  return(p)
}

d_est <- data.frame.quantile.mcmc(x=X_new, y_mcmc=ms$y_base_new)
p <- ggplot.5quantile(data=d_est)
p <- p + geom_point(data=d, aes(x=X, y=Y), shape=1, size=3)
p <- customize.ggplot.axis(p)
ggsave(file='output/fig4-8-left-2.png', plot=p, dpi=300, w=4, h=3)

d_est <- data.frame.quantile.mcmc(x=X_new, y_mcmc=ms$y_new)
p <- ggplot.5quantile(data=d_est)
p <- p + geom_point(data=d, aes(x=X, y=Y), shape=1, size=3)
p <- customize.ggplot.axis(p)
ggsave(file='output/fig4-8-right-2.png', plot=p, dpi=300, w=4, h=3)

- 基本年収のベイズ信頼区間
<img width="400" src="./output/fig4-8-left-2.png">
- 年収のベイズ予測区間
<img width="400" src="./output/fig4-8-right-2.png">

In [18]:
head(ms$y_new[,1:4]) # ms$y_newは [MCMCのサンプル長さ] x  [y_newの添え字23, 24, ...]

281.1099,469.0363,126.4955,573.1011
472.8970,370.5866,503.4453,538.7360
460.3009,309.4866,297.2961,461.3674
497.0330,517.5797,388.8582,601.4932
387.2410,276.0752,435.1163,495.7872
372.2900,461.9367,420.8332,532.4309


---
## 練習問題

In [19]:
# generate-data.R
set.seed(123)
N1 <- 30
N2 <- 20
Y1 <- rnorm(n=N1, mean=0, sd=5)
Y2 <- rnorm(n=N2, mean=1, sd=4)

### (1) 図を書く

In [20]:
# ex1.R
d1 <- data.frame(group=1, Y=Y1)
d2 <- data.frame(group=2, Y=Y2)
d <- rbind(d1, d2)
d$group <- as.factor(d$group)

p <- ggplot(data=d, aes(x=group, y=Y, group=group, col=group))
p <- p + geom_boxplot(outlier.size=0)
p <- p + geom_point(position=position_jitter(w=0.4, h=0), size=2)
ggsave(file='exercise/fig-ex1.png', plot=p, dpi=300, w=4, h=3)

head(d)
tail(d)

group,Y
1,-2.8023782
1,-1.1508874
1,7.7935416
1,0.3525420
1,0.6464387
1,8.5753249


,group,Y
45,2,5.8318480
46,2,-3.4924343
47,2,-0.6115393
48,2,-0.8666214
49,2,4.1198605
50,2,0.6665237


<img width="400" src="exercise/fig-ex1.png"></img>

### (2) モデル式
$$
Y_1[n] \sim Normal(\mu_1, \sigma) \:\:\: n=1, 2, ..., N_1 \\
Y_2[n] \sim Normal(\mu_2, \sigma) \:\:\: n=1, 2, ..., N_2
$$

### (3) モデルファイル
exercise/my_ex3.stan

### (4) MCMCサンプルから$Prob[\mu1 < \mu2]$をR側で計算

In [21]:
data <- list(N1=N1, N2=N2, Y1=Y1, Y2=Y2)
data

$N1
[1] 30

$N2
[1] 20

$Y1
 [1] -2.8023782 -1.1508874  7.7935416  0.3525420  0.6464387  8.5753249
 [7]  2.3045810 -6.3253062 -3.4342643 -2.2283099  6.1204090  1.7990691
[13]  2.0038573  0.5534136 -2.7792057  8.9345657  2.4892524 -9.8330858
[19]  3.5067795 -2.3639570 -5.3391185 -1.0898746 -5.1300222 -3.6444561
[25] -3.1251963 -8.4334666  4.1889352  0.7668656 -5.6906847  6.2690746

$Y2
 [1]  2.7058569 -0.1802859  4.5805026  4.5125340  4.2863243  3.7545610
 [7]  3.2156706  0.7523532 -0.2238507 -0.5218840 -1.7788279  0.1683309
[13] -4.0615854  9.6758239  5.8318480 -3.4924343 -0.6115393 -0.8666214
[19]  4.1198605  0.6665237

In [22]:
data <- list(N1=N1, N2=N2, Y1=Y1, Y2=Y2)
fit <- stan(file='exercise/my_ex3.stan', data=data, seed=1234)
fit

Inference for Stan model: my_ex3.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

        mean se_mean   sd    2.5%    25%    50%    75%  97.5% n_eff Rhat
mu1    -0.24    0.01 0.80   -1.83  -0.77  -0.23   0.29   1.34  3539    1
mu2     1.62    0.02 1.02   -0.38   0.95   1.63   2.28   3.61  3687    1
sigma   4.48    0.01 0.47    3.67   4.16   4.45   4.76   5.53  3396    1
lp__  -97.75    0.03 1.26 -101.02 -98.31 -97.41 -96.82 -96.32  1744    1

Samples were drawn using NUTS(diag_e) at Sat Nov 25 15:52:38 2017.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).

In [23]:
ms <- rstan::extract(fit)
df <- data.frame(mu1=ms$mu1, mu2=ms$mu2)
head(df)

mu1,mu2
0.5650101,0.1577145
-0.4902248,1.4141500
-1.1471710,3.5704500
-1.0820169,1.1194888
0.7643512,4.0232341
0.3133074,2.8117053


In [24]:
cnt = count(df, mu1<mu2)
cnt
# Prob
cnt$n[2] / sum(cnt$n)

mu1 < mu2,n
FALSE,303
TRUE,3697


[1] 0.92425

In [25]:
# ex4.R
# 正解
prob <- mean(ms$mu1 < ms$mu2)  #=> 0.9325
prob

[1] 0.92425

### (5) グループごとの標準偏差が異なる場合
- モデル式
$$
Y_1[n] \sim Normal(\mu_1, \sigma_1) \:\:\: n=1, 2, ..., N_1 \\
Y_2[n] \sim Normal(\mu_2, \sigma_2) \:\:\: n=1, 2, ..., N_2
$$

- モデルファイル
exercise/my_ex5.stan

- $Prob[\mu1 < \mu2]$

In [26]:
data <- list(N1=N1, N2=N2, Y1=Y1, Y2=Y2)
fit <- stan(file='exercise/my_ex5.stan', data=data, seed=1234)
ms <- rstan::extract(fit)

recompiling to avoid crashing R session


In file included from file69b42425bde8.cpp:8:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/src/stan/model/model_header.hpp:4:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/stan/math.hpp:4:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/stan/math/rev/mat.hpp:4:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/stan/math/rev/core.hpp:12:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/stan/math/rev/core/gevv_vvv_vari.hpp:5:
In file included from /usr/local/lib/R/3.4/site-library/StanHeaders/include/stan/math/rev/core/var.hpp:7:
In file included from /usr/local/lib/R/3.4/site-library/BH/include/boost/math/tools/config.hpp:13:
In file included from /usr/local/lib/R/3.4/site-library/BH/include/boost/config.hpp:39:
/usr/local/lib/R/3.4/site-library/BH/include/boost/config/compiler/clang.hpp:200:11: warning: 'BOOST_NO_CXX11_RVALUE_REFERENCES' macro re

In [27]:
prob <- mean(ms$mu1 < ms$mu2)
prob

[1] 0.9365

In [28]:
# 標準偏差を確認してみる
mean(ms$sigma1)
mean(ms$sigma2)

[1] 5.120532

[1] 3.63551